In [4]:
import re
import requests
import tldextract

# Liste de mots-clés suspects souvent utilisés dans le phishing
SUSPICIOUS_KEYWORDS = ["login", "verify", "update", "secure", "bank", "account", "free", "gift", "password"]

# Fonction pour vérifier si l'URL contient une adresse IP au lieu d'un nom de domaine
def is_ip_url(url):
    return bool(re.match(r'^(http|https):\/\/\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', url))

# Fonction pour vérifier si l'URL contient des mots-clés suspects
def contains_suspicious_keywords(url):
    return any(keyword in url.lower() for keyword in SUSPICIOUS_KEYWORDS)

# Fonction pour vérifier si une URL est listée comme malveillante sur VirusTotal
def check_blacklist(url):
    API_KEY = "1d036ad8dc94541e4c730c95b08b027344c47372059f5c3e7100cdd8b20beb6b"  # Remplace par clé API
    vt_url = "https://www.virustotal.com/api/v3/urls"
    headers = {"x-apikey": API_KEY}
    
    try:
        response = requests.post(vt_url, headers=headers, data={"url": url})
        data = response.json()
        
        if "data" in data:
            stats = data["data"]["attributes"]["last_analysis_stats"]
            return stats
    except Exception as e:
        print(f"[ERROR] VirusTotal check failed: {e}")
    
    return None

# Fonction principale d'analyse d'URL
def analyze_url(url):
    domain_info = tldextract.extract(url)
    domain = f"{domain_info.domain}.{domain_info.suffix}"

    print(f"\n🔍 [INFO] Analyse de l'URL : {url}")
    print(f"🌐 Domaine extrait : {domain}")

    if is_ip_url(url):
        print("⚠️ [ALERTE] L'URL utilise une adresse IP au lieu d'un domaine. Risque de phishing.")

    if contains_suspicious_keywords(url):
        print("⚠️ [ALERTE] L'URL contient des mots-clés suspects !")

    blacklist_result = check_blacklist(url)
    if blacklist_result:
        print(f"🛑 [INFO] Résultats VirusTotal : {blacklist_result}")

    print("\n✅ Analyse terminée.\n")

# Exemple d'utilisation
if __name__ == "__main__":
    test_url = input("🔗 Entrez une URL à analyser : ")
    analyze_url(test_url)


🔗 Entrez une URL à analyser : http://192.168.1.1/login

🔍 [INFO] Analyse de l'URL : http://192.168.1.1/login
🌐 Domaine extrait : 192.168.1.1.
⚠️ [ALERTE] L'URL utilise une adresse IP au lieu d'un domaine. Risque de phishing.
⚠️ [ALERTE] L'URL contient des mots-clés suspects !
[ERROR] VirusTotal check failed: 'attributes'

✅ Analyse terminée.



In [ ]:
✅ URL légitime :
🔗 https://www.google.com
🔗 https://www.paypal.com
🔗 https://www.banque-france.fr

⚠️ URL suspectes (exemples fictifs de phishing) :
🔗 http://192.168.1.1/login (Utilise une adresse IP au lieu d'un domaine)
🔗 https://secure-paypal.com-login.verification.xyz (Semble imiter PayPal mais n'est pas officiel)
🔗 http://bank-verification-freegift.com (Contient "bank", "verify" et "free gift")
🔗 https://update-my-password-login.com (Plusieurs mots-clés suspects)